<a href="https://colab.research.google.com/github/tatkoch/MOSiR_Lancut/blob/main/MOSiR_wykresy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import plotly.graph_objects as go

# Wczytaj dane z GitHub
url = "https://raw.githubusercontent.com/tatkoch/MOSiR_Lancut/main/frekwencja.csv"
df = pd.read_csv(url, skiprows=1, names=["timestamp", "aktualna", "maksymalna", "temperatura", "warunki", "prognoza"])

# Parsowanie i czyszczenie danych
df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
df["aktualna"] = pd.to_numeric(df["aktualna"], errors="coerce")
df.dropna(subset=["timestamp", "aktualna"], inplace=True)
df = df[df["aktualna"] > 0]

# Kolumny pomocnicze
df["data"] = df["timestamp"].dt.date
df["godzina"] = df["timestamp"].dt.hour
df["dzien_en"] = df["timestamp"].dt.day_name()

# Tłumaczenie dni tygodnia na polski
dni_polskie = {
    "Monday": "Poniedziałek",
    "Tuesday": "Wtorek",
    "Wednesday": "Środa",
    "Thursday": "Czwartek",
    "Friday": "Piątek",
    "Saturday": "Sobota",
    "Sunday": "Niedziela"
}
df["dzien_pl"] = df["dzien_en"].map(dni_polskie)

dni_kolejnosc = list(dni_polskie.values())
fig = go.Figure()
linie_dzien = []

# Dodaj ślady dla każdego dnia tygodnia
for dzien in dni_kolejnosc:
    df_dzien = df[df["dzien_pl"] == dzien]
    for data in sorted(df_dzien["data"].unique()):
        df_data = df_dzien[df_dzien["data"] == data]
        fig.add_trace(go.Scatter(
            x=df_data["godzina"],
            y=df_data["aktualna"],
            mode="lines+markers",
            name=f"{data} ({dzien})",
            visible=False
        ))
        linie_dzien.append((dzien, data))

# Linie poziome: 80 i 70
godziny = list(range(6, 23))
fig.add_trace(go.Scatter(
    x=godziny,
    y=[80]*len(godziny),
    mode="lines",
    name="Maksymalna (80)",
    line=dict(color="red", dash="dash"),
    visible=True
))
fig.add_trace(go.Scatter(
    x=godziny,
    y=[70]*len(godziny),
    mode="lines",
    name="Ryzykowna (70)",
    line=dict(color="orange", dash="dot"),
    visible=True
))

# Dropdown: dni tygodnia + wszystkie
buttons = []
offset = 0
for dzien in dni_kolejnosc:
    count = sum(1 for d, _ in linie_dzien if d == dzien)
    visible = [False] * len(fig.data)
    for i in range(offset, offset + count):
        visible[i] = True
    visible[-2] = True
    visible[-1] = True
    buttons.append(dict(label=dzien, method="update", args=[{"visible": visible}, {"title": f"Frekwencja — {dzien}"}]))
    offset += count

visible_all = [True] * (len(fig.data) - 2) + [True, True]
buttons.append(dict(label="Wszystkie dni", method="update", args=[{"visible": visible_all}, {"title": "Frekwencja — Wszystkie dni"}]))

fig.update_layout(
    title="Frekwencja — Wszystkie dni",
    xaxis=dict(
        tickmode="array",
        tickvals=list(range(6, 23)),
        title="Godzina",
        range=[6, 22]
    ),
    yaxis_title="Liczba osób",
    updatemenus=[dict(active=len(buttons)-1, buttons=buttons, x=0.1, y=1.15)],
    height=600
)

fig.show()


In [4]:
import pandas as pd
import plotly.graph_objects as go

# Wczytaj dane z GitHub
url = "https://raw.githubusercontent.com/tatkoch/MOSiR_Lancut/main/frekwencja.csv"
df = pd.read_csv(url, skiprows=1, names=["timestamp", "aktualna", "maksymalna", "temperatura", "warunki", "prognoza"])

# Parsowanie i czyszczenie danych
df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
df["aktualna"] = pd.to_numeric(df["aktualna"], errors="coerce")
df.dropna(subset=["timestamp", "aktualna"], inplace=True)
df = df[df["aktualna"] > 0]

# Kolumny pomocnicze
df["godzina"] = df["timestamp"].dt.hour
df["dzien_en"] = df["timestamp"].dt.day_name()

dni_polskie = {
    "Monday": "Poniedziałek",
    "Tuesday": "Wtorek",
    "Wednesday": "Środa",
    "Thursday": "Czwartek",
    "Friday": "Piątek",
    "Saturday": "Sobota",
    "Sunday": "Niedziela"
}
df["dzien_pl"] = df["dzien_en"].map(dni_polskie)
dni_kolejnosc = list(dni_polskie.values())

fig = go.Figure()

# Dodaj średnie dla każdego dnia
for i, dzien in enumerate(dni_kolejnosc):
    df_dzien = df[df["dzien_pl"] == dzien]
    srednia = df_dzien.groupby("godzina")["aktualna"].mean().reindex(range(6, 23))
    fig.add_trace(go.Scatter(
        x=srednia.index,
        y=srednia.values,
        mode="lines+markers",
        name=dzien,
        visible=(i == 0)
    ))

# Linie poziome: 80 i 70
godziny = list(range(6, 23))
fig.add_trace(go.Scatter(
    x=godziny,
    y=[80]*len(godziny),
    mode="lines",
    name="Maksymalna (80)",
    line=dict(color="red", dash="dash"),
    visible=True
))
fig.add_trace(go.Scatter(
    x=godziny,
    y=[70]*len(godziny),
    mode="lines",
    name="Ryzykowna (70)",
    line=dict(color="orange", dash="dot"),
    visible=True
))

# Dropdown
buttons = []
for i, dzien in enumerate(dni_kolejnosc):
    visible = [False] * len(fig.data)
    visible[i] = True
    visible[-2] = True
    visible[-1] = True
    buttons.append(dict(label=dzien, method="update", args=[{"visible": visible}, {"title": f"Średnia frekwencja — {dzien}"}]))

visible_all = [True] * len(dni_kolejnosc) + [True, True]
buttons.append(dict(label="Wszystkie dni", method="update", args=[{"visible": visible_all}, {"title": "Średnia frekwencja — Wszystkie dni"}]))

fig.update_layout(
    title="Średnia frekwencja — Wszystkie dni",
    xaxis=dict(
        tickmode="array",
        tickvals=list(range(6, 23)),
        title="Godzina",
        range=[6, 22]
    ),
    yaxis_title="Liczba osób",
    updatemenus=[dict(active=len(buttons)-1, buttons=buttons, x=0.1, y=1.15)],
    height=600
)

fig.show()


In [5]:

# (Opcjonalnie) W niektórych środowiskach Colab trzeba włączyć menedżera widgetów:
from google.colab import output
output.enable_custom_widget_manager()


import pandas as pd
import plotly.graph_objects as go
import datetime as dt

# ---- Widgets / Colab ----
import ipywidgets as widgets
from IPython.display import display, clear_output

# Wczytaj dane z GitHub
url = "https://raw.githubusercontent.com/tatkoch/MOSiR_Lancut/main/frekwencja.csv"
df = pd.read_csv(url, skiprows=1, names=["timestamp", "aktualna", "maksymalna", "temperatura", "warunki", "prognoza"])

# Parsowanie i czyszczenie danych
df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
df["aktualna"] = pd.to_numeric(df["aktualna"], errors="coerce")
df.dropna(subset=["timestamp", "aktualna"], inplace=True)
df = df[df["aktualna"] > 0]  # poprawione: > zamiast &gt;

# Kolumny pomocnicze
df["data"] = df["timestamp"].dt.date
df["godzina"] = df["timestamp"].dt.hour
df["dzien_en"] = df["timestamp"].dt.day_name()
df["rok"] = df["timestamp"].dt.year
df["miesiac_num"] = df["timestamp"].dt.month

# Tłumaczenie dni tygodnia na polski
dni_polskie = {
    "Monday": "Poniedziałek",
    "Tuesday": "Wtorek",
    "Wednesday": "Środa",
    "Thursday": "Czwartek",
    "Friday": "Piątek",
    "Saturday": "Sobota",
    "Sunday": "Niedziela"
}
df["dzien_pl"] = df["dzien_en"].map(dni_polskie)

# Mapowanie nazw miesięcy (polskie, do dropdownu i etykiet)
miesiace_pl = {
    1: "Styczeń", 2: "Luty", 3: "Marzec", 4: "Kwiecień",
    5: "Maj", 6: "Czerwiec", 7: "Lipiec", 8: "Sierpień",
    9: "Wrzesień", 10: "Październik", 11: "Listopad", 12: "Grudzień"
}
df["miesiac_pl"] = df["miesiac_num"].map(miesiace_pl)

# Kolejność dni tygodnia (jak wymagasz)
dni_kolejnosc = [
    "Poniedziałek", "Wtorek", "Środa", "Czwartek", "Piątek", "Sobota", "Niedziela"
]

# Godziny na osi X i linie odniesienia
godziny = list(range(6, 23))
LINIE_H = [
    dict(y=80, name="Maksymalna (80)", color="red", dash="dash"),
    dict(y=70, name="Ryzykowna (70)", color="orange", dash="dot"),
]

def zbuduj_wykres(dane: pd.DataFrame, tytul: str = "Frekwencja — Wszystkie dni"):
    """Buduje wykres Plotly z dropdownem dni tygodnia, z danych już przefiltrowanych."""
    fig = go.Figure()
    linie_dzien = []

    # Dodaj ślady dla każdego dnia tygodnia w ustalonej kolejności
    for dzien in dni_kolejnosc:
        df_dzien = dane[dane["dzien_pl"] == dzien]
        # unikalne daty tego dnia, posortowane rosnąco
        for data_ in sorted(df_dzien["data"].unique()):
            df_data = df_dzien[df_dzien["data"] == data_].sort_values("godzina")
            fig.add_trace(go.Scatter(
                x=df_data["godzina"],
                y=df_data["aktualna"],
                mode="lines+markers",
                name=f"{data_} ({dzien})",
                visible=False,
                hovertemplate="Godzina: %{x}<br>Osób: %{y}<extra></extra>"
            ))
            linie_dzien.append((dzien, data_))

    # Linie poziome 80 i 70
    for linia in LINIE_H:
        fig.add_trace(go.Scatter(
            x=godziny,
            y=[linia["y"]] * len(godziny),
            mode="lines",
            name=linia["name"],
            line=dict(color=linia["color"], dash=linia["dash"]),
            visible=True
        ))

    # Dropdown: dni tygodnia + wszystkie
    buttons = []
    offset = 0
    for dzien in dni_kolejnosc:
        # ile linii należy do tego dnia
        count = sum(1 for d, _ in linie_dzien if d == dzien)
        visible = [False] * len(fig.data)
        # włącz ślady dla wybranego dnia
        for i in range(offset, offset + count):
            visible[i] = True
        # zawsze włącz linie referencyjne na końcu
        visible[-2] = True
        visible[-1] = True
        buttons.append(dict(
            label=dzien,
            method="update",
            args=[{"visible": visible}, {"title": f"Frekwencja — {dzien}"}]
        ))
        offset += count

    # Przycisk "Wszystkie dni"
    visible_all = [True] * (len(fig.data) - 2) + [True, True]
    buttons.append(dict(label="Wszystkie dni", method="update",
                        args=[{"visible": visible_all}, {"title": "Frekwencja — Wszystkie dni"}]))

    fig.update_layout(
        title=tytul,
        xaxis=dict(
            tickmode="array",
            tickvals=godziny,
            title="Godzina",
            range=[6, 22]
        ),
        yaxis_title="Liczba osób",
        updatemenus=[dict(active=len(buttons)-1, buttons=buttons, x=0.1, y=1.15)],
        height=600,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    # Jeśli w filtrze nie ma żadnych śladów (np. brak danych), ukryj wszystkie oprócz linii odniesienia
    if len(linie_dzien) == 0:
        # Schowaj wszystkie ślady (oprócz dwóch ostatnich – linie odniesienia)
        visible_none = [False] * (len(fig.data) - 2) + [True, True]
        fig.update_layout(updatemenus=[dict(active=len(buttons)-1, buttons=[
            dict(label="Brak danych", method="update",
                 args=[{"visible": visible_none}, {"title": f"{tytul} (brak danych)"}]),
            # zachowaj też przycisk 'Wszystkie dni' do spójności interfejsu
            dict(label="Wszystkie dni", method="update",
                 args=[{"visible": visible_none}, {"title": f"{tytul} (brak danych)"}]),
        ], x=0.1, y=1.15)])

    return fig

# ---- Dropdowny ROK/MIESIĄC (niezależne) ----

# Opcje roku
lata = sorted(df["rok"].unique().tolist())
opcja_wszystkie_lata = "Wszystkie lata"
opcje_roku = [opcja_wszystkie_lata] + lata

# Opcje miesiąca (po polsku)
unik_mies = sorted(df["miesiac_num"].unique().tolist())
opcja_wszystkie_mies = "Wszystkie miesiące"
opcje_mies = [opcja_wszystkie_mies] + [miesiace_pl[m] for m in unik_mies]

# Widgety
dd_rok = widgets.Dropdown(
    options=opcje_roku,
    value=opcja_wszystkie_lata,
    description="Rok:",
    layout=widgets.Layout(width="220px")
)

dd_miesiac = widgets.Dropdown(
    options=opcje_mies,
    value=opcja_wszystkie_mies,
    description="Miesiąc:",
    layout=widgets.Layout(width="220px")
)

# Kontener na wynik (wykres)
out = widgets.Output()

def filtruj_df():
    """Zwraca przefiltrowany DataFrame wg obecnie wybranych roku/miesiąca."""
    filtr = df.copy()

    # Rok
    if isinstance(dd_rok.value, int):
        filtr = filtr[filtr["rok"] == dd_rok.value]

    # Miesiąc
    if dd_miesiac.value != opcja_wszystkie_mies:
        # znajdź numer miesiąca po polskiej nazwie
        mies_num = {v: k for k, v in miesiace_pl.items()}[dd_miesiac.value]
        filtr = filtr[filtr["miesiac_num"] == mies_num]

    return filtr

def aktualizuj_wykres(_=None):
    """Aktualizuje wykres po zmianie roku/miesiąca."""
    dane = filtruj_df()

    # Ustal tytuł na bazie filtrów (dla czytelności)
    tytul_czesci = []
    if isinstance(dd_rok.value, int):
        tytul_czesci.append(str(dd_rok.value))
    else:
        tytul_czesci.append("Wszystkie lata")

    if dd_miesiac.value != opcja_wszystkie_mies:
        tytul_czesci.append(dd_miesiac.value)
    else:
        tytul_czesci.append("Wszystkie miesiące")

    tytul = "Frekwencja — " + ", ".join(tytul_czesci)

    fig = zbuduj_wykres(dane, tytul=tytul)

    with out:
        clear_output(wait=True)
        display(fig)

# Podłącz obsługę zmian
dd_rok.observe(aktualizuj_wykres, names='value')
dd_miesiac.observe(aktualizuj_wykres, names='value')

# Pierwsze wyświetlenie
display(widgets.HBox([dd_rok, dd_miesiac]))
aktualizuj_wykres()
display(out)


Output()